In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer

#########################################################################

# Función que nos da Informacion General del df
def info(df):
     print(df.dtypes)
     print('')
     print(df.isnull().sum())
     print('')
     print(df.select_dtypes(['object']).columns)
     print('')
     for col in df.select_dtypes(['object']).columns:
       print(col, '\n')
       print('Cantidad de valores unicos: ', df[col].nunique())
       if df[col].nunique()<15:
         print(df[col].value_counts())
         print('----------------\n')

# Función para eliminar caracter de una columna
def remove_character(df, column_name, character):
    df[column_name] = df[column_name].str.replace(character, "")
    return df

# Función para convertir una columna categórica en valores numericos
def convert_categorical(df, column_name, option):
    df_copy = df.copy()
    column_data = df_copy[column_name]
    if option == "Dummies":  # Recomendado Logistica, Esta codificación se utiliza cuando tienes una columna "Color" con valores "Rojo", "Verde" y "Azul"
        column_encoded = pd.get_dummies(column_data, prefix=column_name)
        df_copy = pd.concat([df_copy.drop(column_name, axis=1), column_encoded], axis=1)
    elif option == "OneHot":  # Recomendado Logistica, Esta codificación se utiliza cuando tienes una columna "Color" con valores "Rojo", "Verde" y "Azul"
        encoder = OneHotEncoder(sparse_output=False)
        encoded_data = encoder.fit_transform(column_data.values.reshape(-1, 1))
        categories = encoder.categories_[0]
        new_columns = [column_name + "_" + str(category) for category in categories]
        df_encoded = pd.DataFrame(encoded_data, columns=new_columns, index=df_copy.index)
        df_copy = pd.concat([df_copy.drop(column_name, axis=1), df_encoded], axis=1)
    elif option == "Ordinal":  # Recomendado Arboles de Decisión, Esta codificación se utiliza cuando hay un orden o jerarquía lógica entre las categorías. Por ejemplo, si tienes una columna "Tamaño" con categorías "Pequeño", "Mediano" y "Grande", puedes asignar los números 1, 2 y 3 respectivamente a cada categoría.
        encoder = OrdinalEncoder()
        encoded_data = encoder.fit_transform(column_data.values.reshape(-1, 1))
        df_copy[column_name] = encoded_data
    return df_copy

# Función que reemplaza un valor seleccionado por su complemento de valor aleatorio de la misma columna)
def replace_value_with_random(df, column_name, value):
    unique_values = df[column_name].unique()  # Obtener los valores únicos de la columna
    unique_values = unique_values[unique_values != value]  # Excluir el valor a reemplazar
    df.loc[df[column_name] == value, column_name] = np.random.choice(unique_values, size=df[column_name].eq(value).sum())
    return df

# Función para aplicar el escalado estándar y robusto a una columna numerica
def apply_scaler(df, column_name, option):
    if option not in ["Standard", "Robust"]:
        raise ValueError("Invalid option. Choose 'Standard' or 'Robust'.")
    scaler = StandardScaler() if option == "Standard" else RobustScaler()
    df_scaled = df.copy()
    df_scaled[column_name] = scaler.fit_transform(df_scaled[[column_name]])
    return df_scaled

# Función para calcular la mediada, media o moda y reemplazar el Nullo
def replace_null_with_median_mean_mode_column(df, column_name, option):
    if option not in ["median", "mean", "mode"]:
        raise ValueError("Invalid option. Choose 'median', 'mean', or 'mode'.")
    if option == "mode":
        mode_value = df[column_name].value_counts().idxmax()
        df_filled = df.copy()
        df_filled[column_name].fillna(mode_value, inplace=True)
    else:
        imputer = SimpleImputer(strategy=option)
        df_filled = df.copy()
        df_filled[column_name] = imputer.fit_transform(df[column_name].values.reshape(-1, 1))
    return df_filled

#########################################################################

# Ejemplo de dataframe
data = {
    'Column1': ['A', 'B', 'C', 'D'],
    'Column2': [1, 2, 3, 4],
    'Column3': [10, 20, 30, 40],
    'Column4': [100, 200, None, 400]
}
df = pd.DataFrame(data)
print('')
print('#########################################################################')
print('')
print('Valores de Campos Objetos')
print(info(df))
print('')
print('#########################################################################')
print('')
print('Ejemplos de Categóricas')
print('')
print("DataFrame Original:")
print(df)
print('')
# Convertir columna "Column1" usando el enfoque "Dummies"
print("Dummies")
df_dummies = convert_categorical(df, "Column1", "Dummies")
print(df_dummies)
print("")
# Convertir columna "Column1" usando el enfoque "OneHot"
print("OneHot")
df_onehot = convert_categorical(df, "Column1", "OneHot")
print(df_onehot)
print("")
# Convertir columna "Column1" usando el enfoque "Ordinal"
print("Ordinal")
df_ordinal = convert_categorical(df, "Column1", "Ordinal")
print(df_ordinal)
print('')
print('#########################################################################')
print('')
print('Ejemplos de Escalado')
print('')
print("DataFrame Original:")
print(df)
print('')
# Aplicar el escalado estándar a la columna 'Column3'
df_standard_scaled = apply_scaler(df, 'Column3', 'Standard')
print("DataFrame escalado con StandardScaler:")
print(df_standard_scaled)
print('')
# Aplicar el escalado robusto a la columna 'Column3'
df_robust_scaled = apply_scaler(df, 'Column3', 'Robust')
print("\nDataFrame escalado con RobustScaler:")
print(df_robust_scaled)
print('')

print('#########################################################################')
print('')
print('Reemplaza e Imputa Mediana, Media o Moda de la Misma Columna en Valores Nulos')
print('')
print("DataFrame Original:")
print(df)
print('')
print('Mediana')
# Reemplazar los valores nulos con la mediana en la columna 'Columna'
df_filled_median = replace_null_with_median_mean_mode_column(df, 'Column4', 'median')
print(df_filled_median)
print('')
print('Media')
# Reemplazar los valores nulos con la media en la columna 'Columna'
df_filled_mean = replace_null_with_median_mean_mode_column(df, 'Column4', 'mean')
print(df_filled_mean)
print('')
print('Moda')
# Reemplazar los valores nulos con la moda en la columna 'Columna'
df_filled_mean = replace_null_with_median_mean_mode_column(df, 'Column4', 'mode')
print(df_filled_mean)
print('#########################################################################')
print('')
print('Elimina Caracter de Columna')
# Eliminar el carácter 'a' de la columna 'Name'
df = remove_character(df, 'Column1', 'D')
print(df)
print('')
print('#########################################################################')
print('')
print('Reemplaza e Imputa Aleatorio de la Misma Columna en Valor Seleccionado')
value_counts = df['Column1'].value_counts()
print(value_counts)
print('')
# Reemplazar el valor 'A' en la columna 'Column1' por uno aleatorio entre 'B' y 'C'
df = replace_value_with_random(df, 'Column1', 'A')
value_counts = df['Column1'].value_counts()
print(value_counts)
print('')
print('#########################################################################')
print('')



#########################################################################

Valores de Campos Objetos
Column1     object
Column2      int64
Column3      int64
Column4    float64
dtype: object

Column1    0
Column2    0
Column3    0
Column4    1
dtype: int64

Index(['Column1'], dtype='object')

Column1 

Cantidad de valores unicos:  4
A    1
B    1
C    1
D    1
Name: Column1, dtype: int64
----------------

None

#########################################################################

Ejemplos de Categóricas

DataFrame Original:
  Column1  Column2  Column3  Column4
0       A        1       10    100.0
1       B        2       20    200.0
2       C        3       30      NaN
3       D        4       40    400.0

Dummies
   Column2  Column3  Column4  Column1_A  Column1_B  Column1_C  Column1_D
0        1       10    100.0          1          0          0          0
1        2       20    200.0          0          1          0          0
2        3       30      NaN          0          0 